資料建置：匯入套件，將 pandas、numpy、keras、matplotlib 匯入 

In [1]:
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, LSTM, TimeDistributed, RepeatVector
from keras.layers.normalization import BatchNormalization
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping, ModelCheckpoint
import matplotlib.pyplot as plt
%matplotlib inline

Using TensorFlow backend.


讀取資料

In [2]:
def readTrain():
    train = pd.read_csv("./SPY.csv")
    return train
train = readTrain()
train.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,2018-09-26,290.910004,292.239990,289.410004,289.880005,289.880005,79739700
1,2018-09-27,290.410004,291.910004,290.100006,290.690002,290.690002,59249500
2,2018-09-28,289.989990,291.279999,289.950012,290.720001,290.720001,70091400
3,2018-10-01,292.109985,292.929993,290.980011,291.730011,291.730011,62078900
4,2018-10-02,291.559998,292.359985,291.140015,291.559998,291.559998,47258200


Augment Features：除了基本資料提供的Features(Open, High, Low, Close, Adj Close, Volume)以外，還可自己增加Features，例如星期幾、幾月、幾號等等

In [3]:
def augFeatures(train):
    train["Date"] = pd.to_datetime(train["Date"])
    train["year"] = train["Date"].dt.year
    train["month"] = train["Date"].dt.month
    train["date"] = train["Date"].dt.day
    train["day"] = train["Date"].dt.dayofweek
    return train
train_Aug = augFeatures(train)
train_Aug.head()

,Date,Open,High,Low,Close,Adj Close,Volume,year,month,date,day
0,2018-09-26,290.910004,292.239990,289.410004,289.880005,289.880005,79739700,2018,9,26,2
1,2018-09-27,290.410004,291.910004,290.100006,290.690002,290.690002,59249500,2018,9,27,3
2,2018-09-28,289.989990,291.279999,289.950012,290.720001,290.720001,70091400,2018,9,28,4
3,2018-10-01,292.109985,292.929993,290.980011,291.730011,291.730011,62078900,2018,10,1,0
4,2018-10-02,291.559998,292.359985,291.140015,291.559998,291.559998,47258200,2018,10,2,1


Normalization：將所有資料做正規化，而由於 Date 是字串非數字，因此先將它 drop 掉

In [4]:
def normalize(train):
    train = train.drop(["Date"], axis=1)
    train_norm = train.apply(lambda x: (x - np.mean(x)) / (np.max(x) - np.min(x)))
    return train_norm
train_norm = normalize(train_Aug)
train_norm.head()

,Open,High,Low,Close,Adj Close,Volume,year,month,date,day
0,0.350063,0.372143,0.356187,0.331110,0.331110,-0.116683,-0.467742,0.210411,0.339785,-0.010081
1,0.341255,0.365842,0.368174,0.345224,0.345224,-0.204991,-0.467742,0.210411,0.373118,0.239919
2,0.333857,0.353812,0.365568,0.345747,0.345747,-0.158265,-0.467742,0.210411,0.406452,0.489919
3,0.371201,0.385318,0.383463,0.363346,0.363346,-0.192797,-0.467742,0.301320,-0.493548,-0.510081
4,0.361513,0.374434,0.386242,0.360384,0.360384,-0.256670,-0.467742,0.301320,-0.460215,-0.260081


Build Training Data：輸入X_train: 利用前30天的Open, High, Low, Close, Adj Close, Volume, month, year, date, day 作為 Features，shape為 (30, 10)
輸出 Y_train: 利用未來 5 天的 Adj Close 作為 Features，shape 為 (5,1)
我們須將資料做位移的展開作為 Training Data，如圖(1)所示。

In [5]:
def buildTrain(train, pastDay=30, futureDay=5):
    X_train, Y_train = [], []
    for i in range(train.shape[0]-futureDay-pastDay):
        X_train.append(np.array(train.iloc[i:i+pastDay]))
        Y_train.append(np.array(train.iloc[i+pastDay:i+pastDay+futureDay]["Adj Close"]))
    return np.array(X_train), np.array(Y_train)
X_train, Y_train = buildTrain(train_norm, 30, 5)
print(X_train[0])
print(Y_train[0])

[[ 0.35006294  0.37214257  0.35618652  0.33111029  0.33111029 -0.11668331
  -0.46774194  0.21041056  0.33978495 -0.01008065]
 [ 0.34125547  0.36584152  0.36817404  0.34522419  0.34522419 -0.20499073
  -0.46774194  0.21041056  0.37311828  0.23991935]
 [ 0.33385694  0.35381163  0.36556817  0.34574691  0.34574691 -0.15826497
  -0.46774194  0.21041056  0.40645161  0.48991935]
 [ 0.37120053  0.38531811  0.38346252  0.36334597  0.36334597 -0.19279675
  -0.46774194  0.30131965 -0.49354839 -0.51008065]
 [ 0.36151254  0.37443386  0.38624229  0.36038355  0.36038355 -0.2566701
  -0.46774194  0.30131965 -0.46021505 -0.26008065]
 [ 0.38229803  0.39066464  0.38936933  0.36317155  0.36317155 -0.18152377
  -0.46774194  0.30131965 -0.42688172 -0.01008065]
 [ 0.35481878  0.35304766  0.32578346  0.3234434   0.3234434   0.02039312
  -0.46774194  0.30131965 -0.39354839  0.23991935]
 [ 0.32857267  0.33452559  0.30076604  0.29521558  0.29521558 -0.00371642
  -0.46774194  0.30131965 -0.36021505  0.48991935]
 

In [6]:
def shuffle(X,Y):
    np.random.seed(10)
    randomList = np.arange(X.shape[0])
    np.random.shuffle(randomList)
    return X[randomList], Y[randomList]
X_train, Y_train = shuffle(X_train, Y_train)

print(X_train[0])
print(Y_train[0])

[[-0.23739545 -0.2642903  -0.2209503  -0.22490985 -0.22490985 -0.05088695
   0.53225806 -0.51686217 -0.22688172 -0.01008065]
 [-0.26029466 -0.25951657 -0.23293782 -0.20905336 -0.20905336 -0.04305486
   0.53225806 -0.51686217 -0.19354839  0.23991935]
 [-0.23528174 -0.26238069 -0.20635689 -0.20731079 -0.20731079 -0.14203147
   0.53225806 -0.51686217 -0.16021505  0.48991935]
 [-0.24972614 -0.2759385  -0.21712817 -0.23484201 -0.23484201 -0.15474477
   0.53225806 -0.51686217 -0.06021505 -0.51008065]
 [-0.23281541 -0.23011027 -0.19280583 -0.18343913 -0.18343913 -0.09311507
   0.53225806 -0.51686217 -0.02688172 -0.26008065]
 [-0.17979479 -0.20585995 -0.14433454 -0.17246152 -0.17246152 -0.12574669
   0.53225806 -0.51686217  0.00645161 -0.01008065]
 [-0.19423863 -0.16862466 -0.15545363 -0.1379611  -0.1379611  -0.04609538
   0.53225806 -0.51686217  0.03978495  0.23991935]
 [-0.10669236 -0.11019429 -0.10263903 -0.07697488 -0.07697488  0.09087632
   0.53225806 -0.51686217  0.07311828  0.48991935]


In [7]:
def splitData(X,Y,rate):
    X_train = X[int(X.shape[0]*rate):]
    Y_train = Y[int(Y.shape[0]*rate):]
    X_val = X[:int(X.shape[0]*rate)]
    Y_val = Y[:int(Y.shape[0]*rate)]
    return X_train, Y_train, X_val, Y_val
X_train, Y_train, X_val, Y_val = splitData(X_train, Y_train, 0.1)

# from 2 dimmension to 3 dimension
Y_train = Y_train[:,np.newaxis]
Y_val = Y_val[:,np.newaxis]
print(Y_train[0])
print(Y_val[0])

[[-0.01285181 -0.00710192 -0.00448833  0.05614959  0.07165723]]
[[0.1439699  0.1505908  0.14710619 0.14501532 0.13595418]]


In [35]:
def buildOneToOneModel(shape):
    model = Sequential()
    model.add(LSTM(10, input_length=shape[1], input_dim=shape[2],return_sequences=True))
    # output shape: (1, 1)
    model.add(TimeDistributed(Dense(1)))    # or use model.add(Dense(1))
    model.compile(loss="mse", optimizer="adam")
    model.summary()
    return model
model = buildOneToOneModel(X_train.shape)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_16 (LSTM)               (None, 1, 10)             840       
_________________________________________________________________
time_distributed_8 (TimeDist (None, 1, 1)              11        
Total params: 851
Trainable params: 851
Non-trainable params: 0
_________________________________________________________________


/home/user/anaconda3/envs/tensorflow/lib/python3.6/site-packages/ipykernel_launcher.py:3: UserWarning: The `input_dim` and `input_length` arguments in recurrent layers are deprecated. Use `input_shape` instead.
  This is separate from the ipykernel package so we can avoid doing imports until
/home/user/anaconda3/envs/tensorflow/lib/python3.6/site-packages/ipykernel_launcher.py:3: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(10, return_sequences=True, input_shape=(1, 10))`
  This is separate from the ipykernel package so we can avoid doing imports until


In [33]:
train = readTrain()
train_Aug = augFeatures(train)
train_norm = normalize(train_Aug)
# change the last day and next day 
X_train, Y_train = buildTrain(train_norm, 1, 1)
X_train, Y_train = shuffle(X_train, Y_train)
X_train, Y_train, X_val, Y_val = splitData(X_train, Y_train, 0.1)

# from 2 dimmension to 3 dimension
Y_train = Y_train[:,np.newaxis]
Y_val = Y_val[:,np.newaxis]


model = buildOneToOneModel(X_train.shape)
callback = EarlyStopping(monitor="loss", patience=10, verbose=1, mode="auto")
model.fit(X_train, Y_train, epochs=1000, batch_size=128, validation_data=(X_val, Y_val), callbacks=[callback])

/home/user/anaconda3/envs/tensorflow/lib/python3.6/site-packages/ipykernel_launcher.py:3: UserWarning: The `input_dim` and `input_length` arguments in recurrent layers are deprecated. Use `input_shape` instead.
  This is separate from the ipykernel package so we can avoid doing imports until
/home/user/anaconda3/envs/tensorflow/lib/python3.6/site-packages/ipykernel_launcher.py:3: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(10, return_sequences=False, input_shape=(1, 10))`
  This is separate from the ipykernel package so we can avoid doing imports until


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_15 (LSTM)               (None, 10)                840       
_________________________________________________________________
dense_15 (Dense)             (None, 1)                 11        
Total params: 851
Trainable params: 851
Non-trainable params: 0
_________________________________________________________________
Train on 110 samples, validate on 12 samples
Epoch 1/1000
110/110 [==============================] - 1s 8ms/step - loss: 0.0366 - val_loss: 0.0560
Epoch 2/1000
110/110 [==============================] - 0s 29us/step - loss: 0.0360 - val_loss: 0.0547
Epoch 3/1000
110/110 [==============================] - 0s 32us/step - loss: 0.0354 - val_loss: 0.0536
Epoch 4/1000
110/110 [==============================] - 0s 38us/step - loss: 0.0348 - val_loss: 0.0524
Epoch 5/1000
110/110 [==============================] - 0s 36us/step - loss: 0.0342 

Epoch 74/1000
110/110 [==============================] - 0s 27us/step - loss: 0.0091 - val_loss: 0.0105
Epoch 75/1000
110/110 [==============================] - 0s 23us/step - loss: 0.0089 - val_loss: 0.0102
Epoch 76/1000
110/110 [==============================] - 0s 21us/step - loss: 0.0087 - val_loss: 0.0099
Epoch 77/1000
110/110 [==============================] - 0s 60us/step - loss: 0.0085 - val_loss: 0.0096
Epoch 78/1000
110/110 [==============================] - 0s 25us/step - loss: 0.0083 - val_loss: 0.0094
Epoch 79/1000
110/110 [==============================] - 0s 22us/step - loss: 0.0082 - val_loss: 0.0091
Epoch 80/1000
110/110 [==============================] - 0s 24us/step - loss: 0.0080 - val_loss: 0.0089
Epoch 81/1000
110/110 [==============================] - 0s 47us/step - loss: 0.0078 - val_loss: 0.0086
Epoch 82/1000
110/110 [==============================] - 0s 34us/step - loss: 0.0076 - val_loss: 0.0084
Epoch 83/1000
110/110 [==============================] - 0s 25us

Epoch 153/1000
110/110 [==============================] - 0s 21us/step - loss: 0.0028 - val_loss: 0.0042
Epoch 154/1000
110/110 [==============================] - 0s 22us/step - loss: 0.0028 - val_loss: 0.0042
Epoch 155/1000
110/110 [==============================] - 0s 22us/step - loss: 0.0028 - val_loss: 0.0042
Epoch 156/1000
110/110 [==============================] - 0s 20us/step - loss: 0.0028 - val_loss: 0.0042
Epoch 157/1000
110/110 [==============================] - 0s 22us/step - loss: 0.0028 - val_loss: 0.0042
Epoch 158/1000
110/110 [==============================] - 0s 22us/step - loss: 0.0028 - val_loss: 0.0042
Epoch 159/1000
110/110 [==============================] - 0s 21us/step - loss: 0.0027 - val_loss: 0.0042
Epoch 160/1000
110/110 [==============================] - 0s 20us/step - loss: 0.0027 - val_loss: 0.0042
Epoch 161/1000
110/110 [==============================] - 0s 20us/step - loss: 0.0027 - val_loss: 0.0042
Epoch 162/1000
110/110 [==============================]

110/110 [==============================] - 0s 27us/step - loss: 0.0025 - val_loss: 0.0040
Epoch 232/1000
110/110 [==============================] - 0s 29us/step - loss: 0.0025 - val_loss: 0.0040
Epoch 233/1000
110/110 [==============================] - 0s 27us/step - loss: 0.0025 - val_loss: 0.0040
Epoch 234/1000
110/110 [==============================] - 0s 27us/step - loss: 0.0025 - val_loss: 0.0040
Epoch 235/1000
110/110 [==============================] - 0s 29us/step - loss: 0.0025 - val_loss: 0.0040
Epoch 236/1000
110/110 [==============================] - 0s 29us/step - loss: 0.0025 - val_loss: 0.0040
Epoch 237/1000
110/110 [==============================] - 0s 28us/step - loss: 0.0025 - val_loss: 0.0040
Epoch 238/1000
110/110 [==============================] - 0s 25us/step - loss: 0.0025 - val_loss: 0.0040
Epoch 239/1000
110/110 [==============================] - 0s 28us/step - loss: 0.0025 - val_loss: 0.0040
Epoch 240/1000
110/110 [==============================] - 0s 34us/step

...

110/110 [==============================] - 0s 25us/step - loss: 0.0022 - val_loss: 0.0038
Epoch 625/1000
110/110 [==============================] - 0s 66us/step - loss: 0.0022 - val_loss: 0.0038
Epoch 626/1000
110/110 [==============================] - 0s 30us/step - loss: 0.0022 - val_loss: 0.0038
Epoch 627/1000
110/110 [==============================] - 0s 29us/step - loss: 0.0022 - val_loss: 0.0038
Epoch 628/1000
110/110 [==============================] - 0s 27us/step - loss: 0.0022 - val_loss: 0.0038
Epoch 629/1000
110/110 [==============================] - 0s 32us/step - loss: 0.0022 - val_loss: 0.0038
Epoch 630/1000
110/110 [==============================] - 0s 25us/step - loss: 0.0022 - val_loss: 0.0038
Epoch 631/1000
110/110 [==============================] - 0s 25us/step - loss: 0.0022 - val_loss: 0.0038
Epoch 632/1000
110/110 [==============================] - 0s 23us/step - loss: 0.0022 - val_loss: 0.0038
Epoch 633/1000
110/110 [==============================] - 0s 22us/step

Epoch 703/1000
110/110 [==============================] - 0s 27us/step - loss: 0.0022 - val_loss: 0.0038
Epoch 704/1000
110/110 [==============================] - 0s 27us/step - loss: 0.0022 - val_loss: 0.0038
Epoch 705/1000
110/110 [==============================] - 0s 25us/step - loss: 0.0022 - val_loss: 0.0038
Epoch 706/1000
110/110 [==============================] - 0s 27us/step - loss: 0.0022 - val_loss: 0.0038
Epoch 707/1000
110/110 [==============================] - 0s 30us/step - loss: 0.0022 - val_loss: 0.0038
Epoch 708/1000
110/110 [==============================] - 0s 33us/step - loss: 0.0022 - val_loss: 0.0038
Epoch 709/1000
110/110 [==============================] - 0s 31us/step - loss: 0.0022 - val_loss: 0.0038
Epoch 710/1000
110/110 [==============================] - 0s 27us/step - loss: 0.0022 - val_loss: 0.0038
Epoch 711/1000
110/110 [==============================] - 0s 28us/step - loss: 0.0022 - val_loss: 0.0038
Epoch 712/1000
110/110 [==============================]

110/110 [==============================] - 0s 30us/step - loss: 0.0022 - val_loss: 0.0038
Epoch 782/1000
110/110 [==============================] - 0s 26us/step - loss: 0.0022 - val_loss: 0.0038
Epoch 783/1000
110/110 [==============================] - 0s 28us/step - loss: 0.0022 - val_loss: 0.0038
Epoch 784/1000
110/110 [==============================] - 0s 24us/step - loss: 0.0021 - val_loss: 0.0038
Epoch 785/1000
110/110 [==============================] - 0s 23us/step - loss: 0.0021 - val_loss: 0.0038
Epoch 786/1000
110/110 [==============================] - 0s 48us/step - loss: 0.0021 - val_loss: 0.0038
Epoch 787/1000
110/110 [==============================] - 0s 25us/step - loss: 0.0021 - val_loss: 0.0038
Epoch 788/1000
110/110 [==============================] - 0s 23us/step - loss: 0.0021 - val_loss: 0.0038
Epoch 789/1000
110/110 [==============================] - 0s 26us/step - loss: 0.0021 - val_loss: 0.0038
Epoch 790/1000
110/110 [==============================] - 0s 29us/step

Epoch 860/1000
110/110 [==============================] - 0s 25us/step - loss: 0.0021 - val_loss: 0.0038
Epoch 861/1000
110/110 [==============================] - 0s 27us/step - loss: 0.0021 - val_loss: 0.0038
Epoch 862/1000
110/110 [==============================] - 0s 28us/step - loss: 0.0021 - val_loss: 0.0038
Epoch 863/1000
110/110 [==============================] - 0s 28us/step - loss: 0.0021 - val_loss: 0.0038
Epoch 864/1000
110/110 [==============================] - 0s 30us/step - loss: 0.0021 - val_loss: 0.0038
Epoch 865/1000
110/110 [==============================] - 0s 26us/step - loss: 0.0021 - val_loss: 0.0038
Epoch 866/1000
110/110 [==============================] - 0s 29us/step - loss: 0.0021 - val_loss: 0.0038
Epoch 867/1000
110/110 [==============================] - 0s 29us/step - loss: 0.0021 - val_loss: 0.0038
Epoch 868/1000
110/110 [==============================] - 0s 32us/step - loss: 0.0021 - val_loss: 0.0038
Epoch 869/1000
110/110 [==============================]

110/110 [==============================] - 0s 25us/step - loss: 0.0021 - val_loss: 0.0038
Epoch 939/1000
110/110 [==============================] - 0s 27us/step - loss: 0.0021 - val_loss: 0.0038
Epoch 940/1000
110/110 [==============================] - 0s 27us/step - loss: 0.0021 - val_loss: 0.0038
Epoch 941/1000
110/110 [==============================] - 0s 26us/step - loss: 0.0021 - val_loss: 0.0038
Epoch 942/1000
110/110 [==============================] - 0s 25us/step - loss: 0.0021 - val_loss: 0.0038
Epoch 943/1000
110/110 [==============================] - 0s 26us/step - loss: 0.0021 - val_loss: 0.0038
Epoch 944/1000
110/110 [==============================] - 0s 27us/step - loss: 0.0021 - val_loss: 0.0038
Epoch 945/1000
110/110 [==============================] - 0s 24us/step - loss: 0.0021 - val_loss: 0.0038
Epoch 946/1000
110/110 [==============================] - 0s 29us/step - loss: 0.0021 - val_loss: 0.0038
Epoch 947/1000
110/110 [==============================] - 0s 30us/step

In [34]:
print(X_train.shape)
print(Y_train.shape)

(110, 1, 10)
(110, 1)


In [12]:
def buildManyToOneModel(shape):
    model = Sequential()
    model.add(LSTM(10, input_length=shape[1], input_dim=shape[2]))
  # output shape: (1, 1)
    model.add(Dense(1))
    model.compile(loss="mse", optimizer="adam")
    model.summary()
    return model
model = buildManyToOneModel(X_train.shape)

/home/user/anaconda3/envs/tensorflow/lib/python3.6/site-packages/ipykernel_launcher.py:3: UserWarning: The `input_dim` and `input_length` arguments in recurrent layers are deprecated. Use `input_shape` instead.
  This is separate from the ipykernel package so we can avoid doing imports until
/home/user/anaconda3/envs/tensorflow/lib/python3.6/site-packages/ipykernel_launcher.py:3: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(10, input_shape=(1, 10))`
  This is separate from the ipykernel package so we can avoid doing imports until


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_3 (LSTM)                (None, 10)                840       
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 11        
Total params: 851
Trainable params: 851
Non-trainable params: 0
_________________________________________________________________


In [26]:
train = readTrain()
train_Aug = augFeatures(train)
train_norm = normalize(train_Aug)
# change the last day and next day 
X_train, Y_train = buildTrain(train_norm, 30, 1)
X_train, Y_train = shuffle(X_train, Y_train)

# because no return sequence, Y_train and Y_val shape must be 2 dimension
X_train, Y_train, X_val, Y_val = splitData(X_train, Y_train, 0.1)

model = buildManyToOneModel(X_train.shape)
callback = EarlyStopping(monitor="loss", patience=10, verbose=1, mode="auto")
model.fit(X_train, Y_train, epochs=1000, batch_size=128, validation_data=(X_val, Y_val), callbacks=[callback])

/home/user/anaconda3/envs/tensorflow/lib/python3.6/site-packages/ipykernel_launcher.py:3: UserWarning: The `input_dim` and `input_length` arguments in recurrent layers are deprecated. Use `input_shape` instead.
  This is separate from the ipykernel package so we can avoid doing imports until
/home/user/anaconda3/envs/tensorflow/lib/python3.6/site-packages/ipykernel_launcher.py:3: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(10, input_shape=(30, 10))`
  This is separate from the ipykernel package so we can avoid doing imports until


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_10 (LSTM)               (None, 10)                840       
_________________________________________________________________
dense_10 (Dense)             (None, 1)                 11        
Total params: 851
Trainable params: 851
Non-trainable params: 0
_________________________________________________________________
Train on 84 samples, validate on 9 samples
Epoch 1/1000
84/84 [==============================] - 1s 9ms/step - loss: 0.0869 - val_loss: 0.1390
Epoch 2/1000
84/84 [==============================] - 0s 263us/step - loss: 0.0829 - val_loss: 0.1319
Epoch 3/1000
84/84 [==============================] - 0s 239us/step - loss: 0.0791 - val_loss: 0.1251
Epoch 4/1000
84/84 [==============================] - 0s 216us/step - loss: 0.0755 - val_loss: 0.1186
Epoch 5/1000
84/84 [==============================] - 0s 257us/step - loss: 0.0721 - val_lo

Epoch 75/1000
84/84 [==============================] - 0s 280us/step - loss: 0.0073 - val_loss: 0.0036
Epoch 76/1000
84/84 [==============================] - 0s 278us/step - loss: 0.0072 - val_loss: 0.0035
Epoch 77/1000
84/84 [==============================] - 0s 285us/step - loss: 0.0070 - val_loss: 0.0035
Epoch 78/1000
84/84 [==============================] - 0s 284us/step - loss: 0.0069 - val_loss: 0.0036
Epoch 79/1000
84/84 [==============================] - 0s 295us/step - loss: 0.0068 - val_loss: 0.0036
Epoch 80/1000
84/84 [==============================] - 0s 291us/step - loss: 0.0067 - val_loss: 0.0036
Epoch 81/1000
84/84 [==============================] - 0s 296us/step - loss: 0.0066 - val_loss: 0.0036
Epoch 82/1000
84/84 [==============================] - 0s 307us/step - loss: 0.0065 - val_loss: 0.0036
Epoch 83/1000
84/84 [==============================] - 0s 314us/step - loss: 0.0064 - val_loss: 0.0037
Epoch 84/1000
84/84 [==============================] - 0s 321us/step - lo

84/84 [==============================] - 0s 287us/step - loss: 0.0034 - val_loss: 0.0039
Epoch 155/1000
84/84 [==============================] - 0s 298us/step - loss: 0.0034 - val_loss: 0.0039
Epoch 156/1000
84/84 [==============================] - 0s 296us/step - loss: 0.0034 - val_loss: 0.0040
Epoch 157/1000
84/84 [==============================] - 0s 269us/step - loss: 0.0034 - val_loss: 0.0040
Epoch 158/1000
84/84 [==============================] - 0s 271us/step - loss: 0.0034 - val_loss: 0.0040
Epoch 159/1000
84/84 [==============================] - 0s 274us/step - loss: 0.0033 - val_loss: 0.0040
Epoch 160/1000
84/84 [==============================] - 0s 266us/step - loss: 0.0033 - val_loss: 0.0040
Epoch 161/1000
84/84 [==============================] - 0s 273us/step - loss: 0.0033 - val_loss: 0.0040
Epoch 162/1000
84/84 [==============================] - 0s 258us/step - loss: 0.0033 - val_loss: 0.0040
Epoch 163/1000
84/84 [==============================] - 0s 257us/step - loss: 0

Epoch 233/1000
84/84 [==============================] - 0s 271us/step - loss: 0.0024 - val_loss: 0.0038
Epoch 234/1000
84/84 [==============================] - 0s 275us/step - loss: 0.0023 - val_loss: 0.0038
Epoch 235/1000
84/84 [==============================] - 0s 264us/step - loss: 0.0023 - val_loss: 0.0038
Epoch 236/1000
84/84 [==============================] - 0s 258us/step - loss: 0.0023 - val_loss: 0.0038
Epoch 237/1000
84/84 [==============================] - 0s 254us/step - loss: 0.0023 - val_loss: 0.0038
Epoch 238/1000
84/84 [==============================] - 0s 253us/step - loss: 0.0023 - val_loss: 0.0038
Epoch 239/1000
84/84 [==============================] - 0s 292us/step - loss: 0.0023 - val_loss: 0.0038
Epoch 240/1000
84/84 [==============================] - 0s 318us/step - loss: 0.0023 - val_loss: 0.0037
Epoch 241/1000
84/84 [==============================] - 0s 272us/step - loss: 0.0023 - val_loss: 0.0037
Epoch 242/1000
84/84 [==============================] - 0s 246us

...

Epoch 628/1000
84/84 [==============================] - 0s 280us/step - loss: 0.0010 - val_loss: 0.0027
Epoch 629/1000
84/84 [==============================] - 0s 247us/step - loss: 0.0010 - val_loss: 0.0027
Epoch 630/1000
84/84 [==============================] - 0s 277us/step - loss: 0.0010 - val_loss: 0.0027
Epoch 631/1000
84/84 [==============================] - 0s 274us/step - loss: 0.0010 - val_loss: 0.0027
Epoch 632/1000
84/84 [==============================] - 0s 282us/step - loss: 0.0010 - val_loss: 0.0027
Epoch 633/1000
84/84 [==============================] - 0s 297us/step - loss: 0.0010 - val_loss: 0.0027
Epoch 634/1000
84/84 [==============================] - 0s 288us/step - loss: 0.0010 - val_loss: 0.0027
Epoch 635/1000
84/84 [==============================] - 0s 255us/step - loss: 0.0010 - val_loss: 0.0027
Epoch 636/1000
84/84 [==============================] - 0s 292us/step - loss: 0.0010 - val_loss: 0.0027
Epoch 637/1000
84/84 [==============================] - 0s 273us

Epoch 705/1000
84/84 [==============================] - 0s 299us/step - loss: 8.7443e-04 - val_loss: 0.0027
Epoch 706/1000
84/84 [==============================] - 0s 258us/step - loss: 8.7277e-04 - val_loss: 0.0027
Epoch 707/1000
84/84 [==============================] - 0s 246us/step - loss: 8.7112e-04 - val_loss: 0.0027
Epoch 708/1000
84/84 [==============================] - 0s 265us/step - loss: 8.6947e-04 - val_loss: 0.0027
Epoch 709/1000
84/84 [==============================] - 0s 314us/step - loss: 8.6783e-04 - val_loss: 0.0027
Epoch 710/1000
84/84 [==============================] - 0s 257us/step - loss: 8.6620e-04 - val_loss: 0.0027
Epoch 711/1000
84/84 [==============================] - 0s 264us/step - loss: 8.6457e-04 - val_loss: 0.0027
Epoch 712/1000
84/84 [==============================] - 0s 251us/step - loss: 8.6295e-04 - val_loss: 0.0027
Epoch 713/1000
84/84 [==============================] - 0s 263us/step - loss: 8.6133e-04 - val_loss: 0.0027
Epoch 714/1000
84/84 [======

Epoch 781/1000
84/84 [==============================] - 0s 316us/step - loss: 7.6192e-04 - val_loss: 0.0027
Epoch 782/1000
84/84 [==============================] - 0s 279us/step - loss: 7.6059e-04 - val_loss: 0.0027
Epoch 783/1000
84/84 [==============================] - 0s 249us/step - loss: 7.5927e-04 - val_loss: 0.0027
Epoch 784/1000
84/84 [==============================] - 0s 262us/step - loss: 7.5794e-04 - val_loss: 0.0027
Epoch 785/1000
84/84 [==============================] - 0s 235us/step - loss: 7.5663e-04 - val_loss: 0.0027
Epoch 786/1000
84/84 [==============================] - 0s 267us/step - loss: 7.5531e-04 - val_loss: 0.0027
Epoch 787/1000
84/84 [==============================] - 0s 268us/step - loss: 7.5400e-04 - val_loss: 0.0027
Epoch 788/1000
84/84 [==============================] - 0s 268us/step - loss: 7.5269e-04 - val_loss: 0.0027
Epoch 789/1000
84/84 [==============================] - 0s 297us/step - loss: 7.5139e-04 - val_loss: 0.0027
Epoch 790/1000
84/84 [======

Epoch 857/1000
84/84 [==============================] - 0s 261us/step - loss: 6.6905e-04 - val_loss: 0.0028
Epoch 858/1000
84/84 [==============================] - 0s 259us/step - loss: 6.6792e-04 - val_loss: 0.0028
Epoch 859/1000
84/84 [==============================] - 0s 266us/step - loss: 6.6679e-04 - val_loss: 0.0028
Epoch 860/1000
84/84 [==============================] - 0s 252us/step - loss: 6.6566e-04 - val_loss: 0.0028
Epoch 861/1000
84/84 [==============================] - 0s 259us/step - loss: 6.6453e-04 - val_loss: 0.0028
Epoch 862/1000
84/84 [==============================] - 0s 268us/step - loss: 6.6341e-04 - val_loss: 0.0028
Epoch 863/1000
84/84 [==============================] - 0s 264us/step - loss: 6.6229e-04 - val_loss: 0.0028
Epoch 864/1000
84/84 [==============================] - 0s 254us/step - loss: 6.6116e-04 - val_loss: 0.0028
Epoch 865/1000
84/84 [==============================] - 0s 284us/step - loss: 6.6005e-04 - val_loss: 0.0028
Epoch 866/1000
84/84 [======

Epoch 933/1000
84/84 [==============================] - 0s 251us/step - loss: 5.8748e-04 - val_loss: 0.0030
Epoch 934/1000
84/84 [==============================] - 0s 337us/step - loss: 5.8645e-04 - val_loss: 0.0030
Epoch 935/1000
84/84 [==============================] - 0s 243us/step - loss: 5.8543e-04 - val_loss: 0.0030
Epoch 936/1000
84/84 [==============================] - 0s 295us/step - loss: 5.8441e-04 - val_loss: 0.0030
Epoch 937/1000
84/84 [==============================] - 0s 244us/step - loss: 5.8339e-04 - val_loss: 0.0030
Epoch 938/1000
84/84 [==============================] - 0s 247us/step - loss: 5.8237e-04 - val_loss: 0.0030
Epoch 939/1000
84/84 [==============================] - 0s 240us/step - loss: 5.8135e-04 - val_loss: 0.0030
Epoch 940/1000
84/84 [==============================] - 0s 285us/step - loss: 5.8034e-04 - val_loss: 0.0030
Epoch 941/1000
84/84 [==============================] - 0s 254us/step - loss: 5.7932e-04 - val_loss: 0.0030
Epoch 942/1000
84/84 [======

In [28]:
print(X_train.shape)
print(Y_train.shape)

(84, 30, 10)
(84, 1)


In [16]:
def buildOneToManyModel(shape):
    model = Sequential()
    model.add(LSTM(10, input_length=shape[1], input_dim=shape[2]))
    # output shape: (5, 1)
    model.add(Dense(1))
    model.add(RepeatVector(5))
    model.compile(loss="mse", optimizer="adam")
    model.summary()
    return model
model = buildOneToManyModel(X_train.shape)

/home/user/anaconda3/envs/tensorflow/lib/python3.6/site-packages/ipykernel_launcher.py:3: UserWarning: The `input_dim` and `input_length` arguments in recurrent layers are deprecated. Use `input_shape` instead.
  This is separate from the ipykernel package so we can avoid doing imports until
/home/user/anaconda3/envs/tensorflow/lib/python3.6/site-packages/ipykernel_launcher.py:3: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(10, input_shape=(30, 10))`
  This is separate from the ipykernel package so we can avoid doing imports until


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_5 (LSTM)                (None, 10)                840       
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 11        
_________________________________________________________________
repeat_vector_1 (RepeatVecto (None, 5, 1)              0         
Total params: 851
Trainable params: 851
Non-trainable params: 0
_________________________________________________________________


In [17]:
train = readTrain()
train_Aug = augFeatures(train)
train_norm = normalize(train_Aug)
# change the last day and next day 
X_train, Y_train = buildTrain(train_norm, 1, 5)
X_train, Y_train = shuffle(X_train, Y_train)
X_train, Y_train, X_val, Y_val = splitData(X_train, Y_train, 0.1)

# from 2 dimmension to 3 dimension
Y_train = Y_train[:,:,np.newaxis]
Y_val = Y_val[:,:,np.newaxis]

model = buildOneToManyModel(X_train.shape)
callback = EarlyStopping(monitor="loss", patience=10, verbose=1, mode="auto")
model.fit(X_train, Y_train, epochs=1000, batch_size=128, validation_data=(X_val, Y_val), callbacks=[callback])

/home/user/anaconda3/envs/tensorflow/lib/python3.6/site-packages/ipykernel_launcher.py:3: UserWarning: The `input_dim` and `input_length` arguments in recurrent layers are deprecated. Use `input_shape` instead.
  This is separate from the ipykernel package so we can avoid doing imports until
/home/user/anaconda3/envs/tensorflow/lib/python3.6/site-packages/ipykernel_launcher.py:3: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(10, input_shape=(1, 10))`
  This is separate from the ipykernel package so we can avoid doing imports until


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_6 (LSTM)                (None, 10)                840       
_________________________________________________________________
dense_6 (Dense)              (None, 1)                 11        
_________________________________________________________________
repeat_vector_2 (RepeatVecto (None, 5, 1)              0         
Total params: 851
Trainable params: 851
Non-trainable params: 0
_________________________________________________________________
Train on 107 samples, validate on 11 samples
Epoch 1/1000
107/107 [==============================] - 1s 5ms/step - loss: 0.0465 - val_loss: 0.0520
Epoch 2/1000
107/107 [==============================] - 0s 35us/step - loss: 0.0458 - val_loss: 0.0516
Epoch 3/1000
107/107 [==============================] - 0s 37us/step - loss: 0.0452 - val_loss: 0.0512
Epoch 4/1000
107/107 [==============================] -

Epoch 73/1000
107/107 [==============================] - 0s 25us/step - loss: 0.0189 - val_loss: 0.0300
Epoch 74/1000
107/107 [==============================] - 0s 24us/step - loss: 0.0187 - val_loss: 0.0297
Epoch 75/1000
107/107 [==============================] - 0s 22us/step - loss: 0.0184 - val_loss: 0.0295
Epoch 76/1000
107/107 [==============================] - 0s 26us/step - loss: 0.0182 - val_loss: 0.0293
Epoch 77/1000
107/107 [==============================] - 0s 24us/step - loss: 0.0179 - val_loss: 0.0290
Epoch 78/1000
107/107 [==============================] - 0s 22us/step - loss: 0.0177 - val_loss: 0.0288
Epoch 79/1000
107/107 [==============================] - 0s 19us/step - loss: 0.0175 - val_loss: 0.0285
Epoch 80/1000
107/107 [==============================] - 0s 20us/step - loss: 0.0172 - val_loss: 0.0283
Epoch 81/1000
107/107 [==============================] - 0s 23us/step - loss: 0.0170 - val_loss: 0.0280
Epoch 82/1000
107/107 [==============================] - 0s 26us

Epoch 152/1000
107/107 [==============================] - 0s 23us/step - loss: 0.0081 - val_loss: 0.0156
Epoch 153/1000
107/107 [==============================] - 0s 24us/step - loss: 0.0081 - val_loss: 0.0155
Epoch 154/1000
107/107 [==============================] - 0s 21us/step - loss: 0.0081 - val_loss: 0.0154
Epoch 155/1000
107/107 [==============================] - 0s 25us/step - loss: 0.0081 - val_loss: 0.0153
Epoch 156/1000
107/107 [==============================] - 0s 24us/step - loss: 0.0080 - val_loss: 0.0152
Epoch 157/1000
107/107 [==============================] - 0s 27us/step - loss: 0.0080 - val_loss: 0.0151
Epoch 158/1000
107/107 [==============================] - 0s 26us/step - loss: 0.0080 - val_loss: 0.0150
Epoch 159/1000
107/107 [==============================] - 0s 24us/step - loss: 0.0079 - val_loss: 0.0150
Epoch 160/1000
107/107 [==============================] - 0s 24us/step - loss: 0.0079 - val_loss: 0.0149
Epoch 161/1000
107/107 [==============================]

107/107 [==============================] - 0s 29us/step - loss: 0.0074 - val_loss: 0.0128
Epoch 231/1000
107/107 [==============================] - 0s 29us/step - loss: 0.0074 - val_loss: 0.0128
Epoch 232/1000
107/107 [==============================] - 0s 31us/step - loss: 0.0074 - val_loss: 0.0127
Epoch 233/1000
107/107 [==============================] - 0s 25us/step - loss: 0.0074 - val_loss: 0.0127
Epoch 234/1000
107/107 [==============================] - 0s 26us/step - loss: 0.0074 - val_loss: 0.0127
Epoch 235/1000
107/107 [==============================] - 0s 34us/step - loss: 0.0074 - val_loss: 0.0127
Epoch 236/1000
107/107 [==============================] - 0s 63us/step - loss: 0.0074 - val_loss: 0.0127
Epoch 237/1000
107/107 [==============================] - 0s 27us/step - loss: 0.0074 - val_loss: 0.0127
Epoch 238/1000
107/107 [==============================] - 0s 25us/step - loss: 0.0074 - val_loss: 0.0127
Epoch 239/1000
107/107 [==============================] - 0s 28us/step

...

Epoch 623/1000
107/107 [==============================] - 0s 28us/step - loss: 0.0066 - val_loss: 0.0125
Epoch 624/1000
107/107 [==============================] - 0s 25us/step - loss: 0.0066 - val_loss: 0.0125
Epoch 625/1000
107/107 [==============================] - 0s 22us/step - loss: 0.0066 - val_loss: 0.0125
Epoch 626/1000
107/107 [==============================] - 0s 24us/step - loss: 0.0066 - val_loss: 0.0125
Epoch 627/1000
107/107 [==============================] - 0s 22us/step - loss: 0.0066 - val_loss: 0.0125
Epoch 628/1000
107/107 [==============================] - 0s 24us/step - loss: 0.0066 - val_loss: 0.0125
Epoch 629/1000
107/107 [==============================] - 0s 24us/step - loss: 0.0066 - val_loss: 0.0125
Epoch 630/1000
107/107 [==============================] - 0s 51us/step - loss: 0.0066 - val_loss: 0.0125
Epoch 631/1000
107/107 [==============================] - 0s 28us/step - loss: 0.0066 - val_loss: 0.0125
Epoch 632/1000
107/107 [==============================]

107/107 [==============================] - 0s 26us/step - loss: 0.0064 - val_loss: 0.0123
Epoch 702/1000
107/107 [==============================] - 0s 24us/step - loss: 0.0064 - val_loss: 0.0123
Epoch 703/1000
107/107 [==============================] - 0s 44us/step - loss: 0.0064 - val_loss: 0.0123
Epoch 704/1000
107/107 [==============================] - 0s 26us/step - loss: 0.0064 - val_loss: 0.0123
Epoch 705/1000
107/107 [==============================] - 0s 32us/step - loss: 0.0064 - val_loss: 0.0123
Epoch 706/1000
107/107 [==============================] - 0s 31us/step - loss: 0.0064 - val_loss: 0.0123
Epoch 707/1000
107/107 [==============================] - 0s 28us/step - loss: 0.0064 - val_loss: 0.0123
Epoch 708/1000
107/107 [==============================] - 0s 28us/step - loss: 0.0064 - val_loss: 0.0123
Epoch 709/1000
107/107 [==============================] - 0s 26us/step - loss: 0.0064 - val_loss: 0.0123
Epoch 710/1000
107/107 [==============================] - 0s 25us/step

Epoch 780/1000
107/107 [==============================] - 0s 29us/step - loss: 0.0063 - val_loss: 0.0120
Epoch 781/1000
107/107 [==============================] - 0s 21us/step - loss: 0.0063 - val_loss: 0.0120
Epoch 782/1000
107/107 [==============================] - 0s 23us/step - loss: 0.0063 - val_loss: 0.0120
Epoch 783/1000
107/107 [==============================] - 0s 26us/step - loss: 0.0063 - val_loss: 0.0120
Epoch 784/1000
107/107 [==============================] - 0s 24us/step - loss: 0.0063 - val_loss: 0.0120
Epoch 785/1000
107/107 [==============================] - 0s 25us/step - loss: 0.0063 - val_loss: 0.0120
Epoch 786/1000
107/107 [==============================] - 0s 28us/step - loss: 0.0063 - val_loss: 0.0120
Epoch 787/1000
107/107 [==============================] - 0s 25us/step - loss: 0.0063 - val_loss: 0.0120
Epoch 788/1000
107/107 [==============================] - 0s 27us/step - loss: 0.0063 - val_loss: 0.0120
Epoch 789/1000
107/107 [==============================]

107/107 [==============================] - 0s 30us/step - loss: 0.0062 - val_loss: 0.0118
Epoch 859/1000
107/107 [==============================] - 0s 26us/step - loss: 0.0062 - val_loss: 0.0118
Epoch 860/1000
107/107 [==============================] - 0s 22us/step - loss: 0.0062 - val_loss: 0.0118
Epoch 861/1000
107/107 [==============================] - 0s 27us/step - loss: 0.0062 - val_loss: 0.0118
Epoch 862/1000
107/107 [==============================] - 0s 25us/step - loss: 0.0062 - val_loss: 0.0118
Epoch 863/1000
107/107 [==============================] - 0s 31us/step - loss: 0.0062 - val_loss: 0.0118
Epoch 864/1000
107/107 [==============================] - 0s 31us/step - loss: 0.0062 - val_loss: 0.0118
Epoch 865/1000
107/107 [==============================] - 0s 29us/step - loss: 0.0062 - val_loss: 0.0118
Epoch 866/1000
107/107 [==============================] - 0s 26us/step - loss: 0.0062 - val_loss: 0.0118
Epoch 867/1000
107/107 [==============================] - 0s 25us/step

Epoch 937/1000
107/107 [==============================] - 0s 29us/step - loss: 0.0060 - val_loss: 0.0117
Epoch 938/1000
107/107 [==============================] - 0s 27us/step - loss: 0.0060 - val_loss: 0.0117
Epoch 939/1000
107/107 [==============================] - 0s 27us/step - loss: 0.0060 - val_loss: 0.0117
Epoch 940/1000
107/107 [==============================] - 0s 30us/step - loss: 0.0060 - val_loss: 0.0117
Epoch 941/1000
107/107 [==============================] - 0s 24us/step - loss: 0.0060 - val_loss: 0.0117
Epoch 942/1000
107/107 [==============================] - 0s 25us/step - loss: 0.0060 - val_loss: 0.0117
Epoch 943/1000
107/107 [==============================] - 0s 29us/step - loss: 0.0060 - val_loss: 0.0117
Epoch 944/1000
107/107 [==============================] - 0s 33us/step - loss: 0.0060 - val_loss: 0.0117
Epoch 945/1000
107/107 [==============================] - 0s 29us/step - loss: 0.0060 - val_loss: 0.0117
Epoch 946/1000
107/107 [==============================]

In [18]:
def buildManyToManyModel(shape):
    model = Sequential()
    model.add(LSTM(10, input_length=shape[1], input_dim=shape[2], return_sequences=True))

  # output shape: (5, 1)
    model.add(TimeDistributed(Dense(1)))
    model.compile(loss="mse", optimizer="adam")
    model.summary()
    return model
model = buildManyToManyModel(X_train.shape)

/home/user/anaconda3/envs/tensorflow/lib/python3.6/site-packages/ipykernel_launcher.py:3: UserWarning: The `input_dim` and `input_length` arguments in recurrent layers are deprecated. Use `input_shape` instead.
  This is separate from the ipykernel package so we can avoid doing imports until
/home/user/anaconda3/envs/tensorflow/lib/python3.6/site-packages/ipykernel_launcher.py:3: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(10, return_sequences=True, input_shape=(1, 10))`
  This is separate from the ipykernel package so we can avoid doing imports until


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_7 (LSTM)                (None, 1, 10)             840       
_________________________________________________________________
time_distributed_3 (TimeDist (None, 1, 1)              11        
Total params: 851
Trainable params: 851
Non-trainable params: 0
_________________________________________________________________


In [19]:
train = readTrain()
train_Aug = augFeatures(train)
train_norm = normalize(train_Aug)
# change the last day and next day 
X_train, Y_train = buildTrain(train_norm, 5, 5)
X_train, Y_train = shuffle(X_train, Y_train)
X_train, Y_train, X_val, Y_val = splitData(X_train, Y_train, 0.1)

# from 2 dimmension to 3 dimension
Y_train = Y_train[:,:,np.newaxis]
Y_val = Y_val[:,:,np.newaxis]

model = buildManyToManyModel(X_train.shape)
callback = EarlyStopping(monitor="loss", patience=10, verbose=1, mode="auto")
model.fit(X_train, Y_train, epochs=1000, batch_size=128, validation_data=(X_val, Y_val), callbacks=[callback])

/home/user/anaconda3/envs/tensorflow/lib/python3.6/site-packages/ipykernel_launcher.py:3: UserWarning: The `input_dim` and `input_length` arguments in recurrent layers are deprecated. Use `input_shape` instead.
  This is separate from the ipykernel package so we can avoid doing imports until
/home/user/anaconda3/envs/tensorflow/lib/python3.6/site-packages/ipykernel_launcher.py:3: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(10, return_sequences=True, input_shape=(5, 10))`
  This is separate from the ipykernel package so we can avoid doing imports until


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_8 (LSTM)                (None, 5, 10)             840       
_________________________________________________________________
time_distributed_4 (TimeDist (None, 5, 1)              11        
Total params: 851
Trainable params: 851
Non-trainable params: 0
_________________________________________________________________
Train on 103 samples, validate on 11 samples
Epoch 1/1000
103/103 [==============================] - 1s 6ms/step - loss: 0.0548 - val_loss: 0.0817
Epoch 2/1000
103/103 [==============================] - 0s 69us/step - loss: 0.0533 - val_loss: 0.0798
Epoch 3/1000
103/103 [==============================] - 0s 71us/step - loss: 0.0517 - val_loss: 0.0781
Epoch 4/1000
103/103 [==============================] - 0s 53us/step - loss: 0.0504 - val_loss: 0.0763
Epoch 5/1000
103/103 [==============================] - 0s 55us/step - loss: 0.0490 

Epoch 74/1000
103/103 [==============================] - 0s 59us/step - loss: 0.0153 - val_loss: 0.0274
Epoch 75/1000
103/103 [==============================] - 0s 51us/step - loss: 0.0153 - val_loss: 0.0273
Epoch 76/1000
103/103 [==============================] - 0s 52us/step - loss: 0.0152 - val_loss: 0.0272
Epoch 77/1000
103/103 [==============================] - 0s 45us/step - loss: 0.0151 - val_loss: 0.0271
Epoch 78/1000
103/103 [==============================] - 0s 45us/step - loss: 0.0150 - val_loss: 0.0270
Epoch 79/1000
103/103 [==============================] - 0s 47us/step - loss: 0.0149 - val_loss: 0.0269
Epoch 80/1000
103/103 [==============================] - 0s 46us/step - loss: 0.0148 - val_loss: 0.0268
Epoch 81/1000
103/103 [==============================] - 0s 48us/step - loss: 0.0148 - val_loss: 0.0267
Epoch 82/1000
103/103 [==============================] - 0s 49us/step - loss: 0.0147 - val_loss: 0.0266
Epoch 83/1000
103/103 [==============================] - 0s 48us

Epoch 153/1000
103/103 [==============================] - 0s 52us/step - loss: 0.0115 - val_loss: 0.0218
Epoch 154/1000
103/103 [==============================] - 0s 60us/step - loss: 0.0115 - val_loss: 0.0218
Epoch 155/1000
103/103 [==============================] - 0s 53us/step - loss: 0.0114 - val_loss: 0.0217
Epoch 156/1000
103/103 [==============================] - 0s 53us/step - loss: 0.0114 - val_loss: 0.0217
Epoch 157/1000
103/103 [==============================] - 0s 49us/step - loss: 0.0114 - val_loss: 0.0216
Epoch 158/1000
103/103 [==============================] - 0s 50us/step - loss: 0.0113 - val_loss: 0.0216
Epoch 159/1000
103/103 [==============================] - 0s 52us/step - loss: 0.0113 - val_loss: 0.0215
Epoch 160/1000
103/103 [==============================] - 0s 54us/step - loss: 0.0113 - val_loss: 0.0215
Epoch 161/1000
103/103 [==============================] - 0s 54us/step - loss: 0.0112 - val_loss: 0.0214
Epoch 162/1000
103/103 [==============================]

103/103 [==============================] - 0s 61us/step - loss: 0.0096 - val_loss: 0.0182
Epoch 232/1000
103/103 [==============================] - 0s 57us/step - loss: 0.0095 - val_loss: 0.0182
Epoch 233/1000
103/103 [==============================] - 0s 51us/step - loss: 0.0095 - val_loss: 0.0182
Epoch 234/1000
103/103 [==============================] - 0s 63us/step - loss: 0.0095 - val_loss: 0.0181
Epoch 235/1000
103/103 [==============================] - 0s 62us/step - loss: 0.0095 - val_loss: 0.0181
Epoch 236/1000
103/103 [==============================] - 0s 54us/step - loss: 0.0095 - val_loss: 0.0180
Epoch 237/1000
103/103 [==============================] - 0s 53us/step - loss: 0.0095 - val_loss: 0.0180
Epoch 238/1000
103/103 [==============================] - 0s 50us/step - loss: 0.0094 - val_loss: 0.0180
Epoch 239/1000
103/103 [==============================] - 0s 52us/step - loss: 0.0094 - val_loss: 0.0179
Epoch 240/1000
103/103 [==============================] - 0s 51us/step

...

Epoch 624/1000
103/103 [==============================] - 0s 56us/step - loss: 0.0064 - val_loss: 0.0138
Epoch 625/1000
103/103 [==============================] - 0s 53us/step - loss: 0.0064 - val_loss: 0.0138
Epoch 626/1000
103/103 [==============================] - 0s 54us/step - loss: 0.0064 - val_loss: 0.0138
Epoch 627/1000
103/103 [==============================] - 0s 53us/step - loss: 0.0064 - val_loss: 0.0138
Epoch 628/1000
103/103 [==============================] - 0s 56us/step - loss: 0.0064 - val_loss: 0.0138
Epoch 629/1000
103/103 [==============================] - 0s 57us/step - loss: 0.0064 - val_loss: 0.0138
Epoch 630/1000
103/103 [==============================] - 0s 53us/step - loss: 0.0063 - val_loss: 0.0138
Epoch 631/1000
103/103 [==============================] - 0s 50us/step - loss: 0.0063 - val_loss: 0.0138
Epoch 632/1000
103/103 [==============================] - 0s 56us/step - loss: 0.0063 - val_loss: 0.0138
Epoch 633/1000
103/103 [==============================]

103/103 [==============================] - 0s 85us/step - loss: 0.0060 - val_loss: 0.0137
Epoch 703/1000
103/103 [==============================] - 0s 57us/step - loss: 0.0060 - val_loss: 0.0137
Epoch 704/1000
103/103 [==============================] - 0s 59us/step - loss: 0.0060 - val_loss: 0.0137
Epoch 705/1000
103/103 [==============================] - 0s 67us/step - loss: 0.0060 - val_loss: 0.0137
Epoch 706/1000
103/103 [==============================] - 0s 56us/step - loss: 0.0060 - val_loss: 0.0137
Epoch 707/1000
103/103 [==============================] - 0s 66us/step - loss: 0.0060 - val_loss: 0.0137
Epoch 708/1000
103/103 [==============================] - 0s 57us/step - loss: 0.0060 - val_loss: 0.0137
Epoch 709/1000
103/103 [==============================] - 0s 52us/step - loss: 0.0060 - val_loss: 0.0137
Epoch 710/1000
103/103 [==============================] - 0s 54us/step - loss: 0.0060 - val_loss: 0.0137
Epoch 711/1000
103/103 [==============================] - 0s 58us/step

Epoch 781/1000
103/103 [==============================] - 0s 56us/step - loss: 0.0056 - val_loss: 0.0135
Epoch 782/1000
103/103 [==============================] - 0s 55us/step - loss: 0.0056 - val_loss: 0.0135
Epoch 783/1000
103/103 [==============================] - 0s 57us/step - loss: 0.0056 - val_loss: 0.0135
Epoch 784/1000
103/103 [==============================] - 0s 59us/step - loss: 0.0056 - val_loss: 0.0135
Epoch 785/1000
103/103 [==============================] - 0s 57us/step - loss: 0.0056 - val_loss: 0.0135
Epoch 786/1000
103/103 [==============================] - 0s 59us/step - loss: 0.0056 - val_loss: 0.0135
Epoch 787/1000
103/103 [==============================] - 0s 55us/step - loss: 0.0056 - val_loss: 0.0135
Epoch 788/1000
103/103 [==============================] - 0s 55us/step - loss: 0.0056 - val_loss: 0.0135
Epoch 789/1000
103/103 [==============================] - 0s 84us/step - loss: 0.0056 - val_loss: 0.0135
Epoch 790/1000
103/103 [==============================]

103/103 [==============================] - 0s 59us/step - loss: 0.0052 - val_loss: 0.0133
Epoch 860/1000
103/103 [==============================] - 0s 56us/step - loss: 0.0052 - val_loss: 0.0132
Epoch 861/1000
103/103 [==============================] - 0s 56us/step - loss: 0.0052 - val_loss: 0.0132
Epoch 862/1000
103/103 [==============================] - 0s 54us/step - loss: 0.0052 - val_loss: 0.0132
Epoch 863/1000
103/103 [==============================] - 0s 54us/step - loss: 0.0052 - val_loss: 0.0132
Epoch 864/1000
103/103 [==============================] - 0s 60us/step - loss: 0.0052 - val_loss: 0.0132
Epoch 865/1000
103/103 [==============================] - 0s 56us/step - loss: 0.0052 - val_loss: 0.0132
Epoch 866/1000
103/103 [==============================] - 0s 57us/step - loss: 0.0052 - val_loss: 0.0132
Epoch 867/1000
103/103 [==============================] - 0s 52us/step - loss: 0.0052 - val_loss: 0.0132
Epoch 868/1000
103/103 [==============================] - 0s 54us/step

Epoch 938/1000
103/103 [==============================] - 0s 53us/step - loss: 0.0049 - val_loss: 0.0129
Epoch 939/1000
103/103 [==============================] - 0s 56us/step - loss: 0.0049 - val_loss: 0.0129
Epoch 940/1000
103/103 [==============================] - 0s 58us/step - loss: 0.0049 - val_loss: 0.0129
Epoch 941/1000
103/103 [==============================] - 0s 56us/step - loss: 0.0049 - val_loss: 0.0129
Epoch 942/1000
103/103 [==============================] - 0s 52us/step - loss: 0.0049 - val_loss: 0.0129
Epoch 943/1000
103/103 [==============================] - 0s 76us/step - loss: 0.0049 - val_loss: 0.0129
Epoch 944/1000
103/103 [==============================] - 0s 50us/step - loss: 0.0049 - val_loss: 0.0129
Epoch 945/1000
103/103 [==============================] - 0s 58us/step - loss: 0.0048 - val_loss: 0.0129
Epoch 946/1000
103/103 [==============================] - 0s 61us/step - loss: 0.0048 - val_loss: 0.0129
Epoch 947/1000
103/103 [==============================]